# Regression Analysis Final Project Data Merging and Cleaning
### Sam Berkson

In [244]:
# Import libraries 
import pandas as pd
import os

# Merging and Cleaning Economic Data

In [245]:
files = ["Recession.csv", "NationalDebt.csv", "MortgageDebtIncomePercent.csv", "MedianSalesPrice.csv", "MortgageLiability.csv", "PersonalIncomePerCapita.csv"]
master = pd.DataFrame()

for index, filename in enumerate(files):
    df = pd.read_csv(os.path.join(os.path.join(os.getcwd(), "Data/"), filename))
    
    if index == 0:
        master = df
    else:
        master = pd.merge(master, df, on="DATE", how="outer")



In [246]:
print("Master Shape: ", master.shape)
print("Master Columns: ", master.columns)

# Set index to date and convert to datetime dtype
master["DATE"] = pd.to_datetime(master["DATE"])


Master Shape:  (328, 7)
Master Columns:  Index(['DATE', 'Recession', 'NationalDebt', 'MortgageDebtIncomePercent',
       'HomeMedianSalesPrice', 'MortgageLiability', 'PersonalIncomePerCapita'],
      dtype='object')


In [247]:
# If date is before 1990 or after 2021, drop it.
master = master.loc[(master["DATE"] > "1990-01-01") & (master["DATE"] < "2021-01-01")]

# Check results
print("Master Shape: ", master.shape)
print("Master Columns: ", master.columns)

Master Shape:  (123, 7)
Master Columns:  Index(['DATE', 'Recession', 'NationalDebt', 'MortgageDebtIncomePercent',
       'HomeMedianSalesPrice', 'MortgageLiability', 'PersonalIncomePerCapita'],
      dtype='object')


In [248]:
# Impute PersonalIncomePerCapita for previous 3 time steps of each entry
master["PersonalIncomePerCapita"] = master["PersonalIncomePerCapita"].fillna(method="bfill", limit=3)

# Impute MedianSalesPrice for next 3 time steps of last entry
master["PersonalIncomePerCapita"] = master["PersonalIncomePerCapita"].fillna(method="ffill", limit=3)

# Check for any nans
print("Number of NaNs: ", master.isna().sum())

Number of NaNs:  DATE                         0
Recession                    0
NationalDebt                 0
MortgageDebtIncomePercent    0
HomeMedianSalesPrice         0
MortgageLiability            0
PersonalIncomePerCapita      0
dtype: int64


In [249]:
# End check
print("Master Shape: ", master.shape)
print("Master Columns: ", master.columns)

# Save to csv
master.to_csv(os.path.join(os.getcwd(), "Data/Master_Data.csv"))

Master Shape:  (123, 7)
Master Columns:  Index(['DATE', 'Recession', 'NationalDebt', 'MortgageDebtIncomePercent',
       'HomeMedianSalesPrice', 'MortgageLiability', 'PersonalIncomePerCapita'],
      dtype='object')


# Cleaning and Merging Poverty Data

In [250]:
df = pd.read_excel(os.path.join(os.path.join(os.getcwd(), "Data/"), "CleanedPovertyByFamilyType.xlsx"))

# Add a year column to both df and master
df["Year"] = df["Year"].astype(str)
df["Year"] = df["Year"].str[:4]
df["Year"] = df["Year"].astype(int)
master["Year"] = master["DATE"].dt.year

In [251]:
# Save master to csv
master = pd.merge(master, df, on="Year", how="left")

print("Master shape: ", master.shape)
print("Master columns: ", master.columns)

Master shape:  (131, 20)
Master columns:  Index(['DATE', 'Recession', 'NationalDebt', 'MortgageDebtIncomePercent',
       'HomeMedianSalesPrice', 'MortgageLiability', 'PersonalIncomePerCapita',
       'Year', 'All_Total', 'All_Num', 'All_Pct', 'Married_Total',
       'Married_Num', 'Married_Pct', 'Male_Total', 'Male_Num', 'Male_Pct',
       'Female_Total', 'Female_Num', 'Female_Pct'],
      dtype='object')


In [252]:
# Remove duplicate DATE entries
master = master.drop_duplicates(subset="DATE", keep="first")

# Remove temp year column
master = master.drop(columns=["Year"])

print("Master shape: ", master.shape)
print("Master columns: ", master.columns)

Master shape:  (123, 19)
Master columns:  Index(['DATE', 'Recession', 'NationalDebt', 'MortgageDebtIncomePercent',
       'HomeMedianSalesPrice', 'MortgageLiability', 'PersonalIncomePerCapita',
       'All_Total', 'All_Num', 'All_Pct', 'Married_Total', 'Married_Num',
       'Married_Pct', 'Male_Total', 'Male_Num', 'Male_Pct', 'Female_Total',
       'Female_Num', 'Female_Pct'],
      dtype='object')


In [253]:
master.to_csv(os.path.join(os.getcwd(), "Data/Master_Data.csv"))